In [1]:
import cv2
import numpy as np
import os

# --- 사용자 설정 ---
normal_image_path = '/content/drive/MyDrive/dataset/atm_dataset/side2_scratch/rotation5/그림316.png'
defect_dir = '/content/drive/MyDrive/result/scratch_128x128_StyleGAN3_result'  # 결함 이미지 폴더
label_path = '/content/drive/MyDrive/dataset/labels/그림316.txt'
output_dir = '/content/drive/MyDrive/sample1'

# --- 출력 디렉토리 생성 ---
os.makedirs(output_dir, exist_ok=True)

# --- 정상 이미지 & 라벨 로딩 ---
background = cv2.imread(normal_image_path)
if background is None:
    raise FileNotFoundError(f"정상 이미지 로딩 실패: {normal_image_path}")

h, w, _ = background.shape
with open(label_path, 'r') as f:
    line = f.readline().strip()
    _, cx, cy, bw, bh = map(float, line.split())

# --- YOLO 좌표 → 픽셀 좌표 변환 ---
x_center = int(cx * w)
y_center = int(cy * h)
box_w = int(bw * w)
box_h = int(bh * h)

# --- 결함 이미지 디렉토리 순회 ---
for fname in os.listdir(defect_dir):
    if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    defect_path = os.path.join(defect_dir, fname)
    defect = cv2.imread(defect_path)
    if defect is None:
        print(f"⚠️ 결함 이미지 로딩 실패: {fname}")
        continue

    # --- 결함 이미지 리사이즈 ---
    resized_defect = cv2.resize(defect, (box_w, box_h))

    # --- 마스크 생성 (흰색) ---
    mask = 255 * np.ones(resized_defect.shape, dtype=resized_defect.dtype)

    # --- 정상 이미지 복사본 생성 ---
    output_img = background.copy()

    # --- Poisson blending 수행 ---
    center = (x_center, y_center)
    blended = cv2.seamlessClone(
        resized_defect,
        output_img,
        mask,
        center,
        cv2.NORMAL_CLONE
    )

    # --- 저장 ---
    base_name = os.path.splitext(fname)[0]
    output_path = os.path.join(output_dir, f"{base_name}_blended.png")
    cv2.imwrite(output_path, blended)
    print(f"✅ 저장 완료: {output_path}")


✅ 저장 완료: /content/drive/MyDrive/sample1/seed0007_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0003_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0005_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0010_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0004_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0000_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0014_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0002_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0011_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0006_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0015_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0009_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0013_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0008_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0012_blended.png
✅ 저장 완료: /content/drive/MyDrive/sample1/seed0001_blended.png
✅ 저장 완료: /content/drive/

In [2]:
import cv2
import numpy as np
import os

# --- 사용자 설정 ---
normal_base_dir = '/content/drive/MyDrive/dataset/atm_dataset/side2_scratch'
defect_dir = '/content/drive/MyDrive/result/scratch_128x128_StyleGAN3_result'
label_dir = '/content/drive/MyDrive/dataset/labels'
output_dir = '/content/drive/MyDrive/sample2'

# --- 출력 디렉토리 생성 ---
os.makedirs(output_dir, exist_ok=True)

# --- 결함 이미지 로딩 ---
defect_files = [f for f in os.listdir(defect_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# --- 모든 하위 이미지 순회 ---
for root, _, files in os.walk(normal_base_dir):
    for fname in files:
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        normal_image_path = os.path.join(root, fname)
        background = cv2.imread(normal_image_path)
        if background is None:
            print(f"⚠️ 정상 이미지 로딩 실패: {normal_image_path}")
            continue

        # 라벨 파일 찾기
        label_name = os.path.splitext(fname)[0] + '.txt'
        label_path = os.path.join(label_dir, label_name)
        if not os.path.exists(label_path):
            print(f"❌ 라벨 파일 없음: {label_path}")
            continue

        h, w, _ = background.shape
        with open(label_path, 'r') as f:
            line = f.readline().strip()
            _, cx, cy, bw, bh = map(float, line.split())

        # --- YOLO 좌표 → 픽셀 변환 ---
        x_center = int(cx * w)
        y_center = int(cy * h)
        box_w = int(bw * w)
        box_h = int(bh * h)

        # --- 결함 이미지들에 대해 반복 ---
        for defect_fname in defect_files:
            defect_path = os.path.join(defect_dir, defect_fname)
            defect = cv2.imread(defect_path)
            if defect is None:
                print(f"⚠️ 결함 이미지 로딩 실패: {defect_fname}")
                continue

            resized_defect = cv2.resize(defect, (box_w, box_h))
            mask = 255 * np.ones(resized_defect.shape, dtype=resized_defect.dtype)
            output_img = background.copy()
            center = (x_center, y_center)

            blended = cv2.seamlessClone(
                resized_defect, output_img, mask, center, cv2.NORMAL_CLONE
            )

            # 저장: 평평하게 한 디렉토리에 저장
            orig_base = os.path.splitext(fname)[0]
            defect_base = os.path.splitext(defect_fname)[0]
            save_name = f"{orig_base}__{defect_base}_blended.png"
            output_path = os.path.join(output_dir, save_name)
            cv2.imwrite(output_path, blended)

KeyboardInterrupt: 